In [1]:
import pandas as pd

Read in the 1920_2024_geopy and 

In [2]:
df = pd.read_json("data/silver/1920_2024_geopy.json")
df[df.Country_location.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_location.csv")
df_ = df[df.Country_location.apply(lambda x: isinstance(x,dict))]
df_[:1]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location
0,1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope..."


In [3]:
df_country_codes = pd.read_json("data/country_codes.json")
df_country_codes[df_country_codes.Country_location.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_codes.csv")
df_country_codes_ = df_country_codes[df_country_codes.Country_location.apply(lambda x: isinstance(x,dict))]
df_country_codes_ = df_country_codes_[df_country_codes_.name.apply(lambda x: x not in ['Palestine, State of', 'Holy See'])]
df_country_codes[:4]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,Country_location
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open..."


In [4]:
len(df_country_codes),len(df_)

(248, 4918)

In [5]:

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def unpack_locations(x,a=-2):
    "Check if is numeric"
    return ",".join([i for i in x['display_name'].split(",") if i.strip().isnumeric()==False][a:]).strip()

def with_conditions(x):
    if has_numbers(unpack_locations(x,a=-2)):
        return unpack_locations(x,a=-1)
    return unpack_locations(x,a=-2)

def get_country_from_dict(df):
    df["Country_Geopy"] = df.Country_location.apply(with_conditions)
    df = df.set_index("Country_Geopy")
    return df

def check_and_compare(dfContryCode,seriesIndex):
    return dfContryCode.loc[seriesIndex]["alpha-3"]


def get_code(country_code,x):
    try: 
        try:
            return  check_and_compare(country_code, x)
        except:
            return check_and_compare(country_code, x.split(",")[-1].strip())
    except:
        return float("nan")

def get_single_data_frame(x):
    return x 


In [6]:
country_code = get_country_from_dict(df_country_codes_)
data_df = get_country_from_dict(df_)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_82324/1916793209.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Country_Geopy"] = df.Country_location.apply(with_conditions)


In [7]:
data_df["Code"]=data_df.index.map(lambda x: get_code(country_code,x))
data_df[:5]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Code
Country_Geopy,,,,,,,,,,,,,,,,,,,
"Northern Ireland, United Kingdom",1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope...",GBR
"Washington, United States",1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope...",USA
"Idaho, United States",1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope...",USA
Italy,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope...",ITA
"Sardinia, Italy",1950,1953-01-26,1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,None,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,"{'total': 19, 'passengers': 15, 'crew': 4}","{'total': 19, 'passengers': 15, 'crew': 4}",0.0,Crashed 10 miles east of Cagliari in the Sinna...,19,19,Linee Aeree Italiane,"{'place_id': 78523394, 'licence': 'Data © Open...",ITA


In [8]:
# 'This country match >>> Holy See >>> Texas, United States', red flag here Holy See is being tide to Texas so drop 
# This list comprehension checks if the name is in the index if it's not in there we list it to check sanity 
def check_index_equal_to_series(sr):
    [print(f"This country match >>> {sr[sr == i].index[0]} >>> {i} ") for i in sr if i not in sr[sr == i].index[0]]
check_index_equal_to_series(country_code["name"])

This country match >>> Caribbean Netherlands, Netherlands >>> Sint Eustatius and Saba Bonaire 
This country match >>> Brunei >>> Brunei Darussalam 
This country match >>> Cape Verde >>> Cabo Verde 
This country match >>> Congo-Brazzaville >>> Republic of the Congo 
This country match >>> Curacao, Netherlands >>> Curaçao 
This country match >>> Falkland Islands >>> Falkland Islands (Malvinas) 
This country match >>> French Southern and Antarctic Lands, France >>> French Southern Territories 
This country match >>> North Korea >>> Korea North 
This country match >>> South Korea >>> Korea South 
This country match >>> Macau, China >>> Macao 
This country match >>> Palestinian Territories >>> Palestine 
This country match >>> Russia >>> Russian Federation 
This country match >>> Saint Martin (France), France >>> Saint Martin (French part) 
This country match >>> São Tomé and Príncipe >>> Sao Tome and Principe 
This country match >>> Syria >>> Syrian Arab Republic 
This country match >>> Ea

In [9]:
check_index_equal_to_series(data_df["Location"])

This country match >>> Northern Ireland, United Kingdom >>> Belfast, Northern Ireland 
This country match >>> Washington, United States >>> Issaquah, Washington 
This country match >>> Idaho, United States >>> Fish Haven, Idaho 
This country match >>> Italy >>> 25 nm off Agrigento, Italy 
This country match >>> Sardinia, Italy >>> Sinnai, Sardinia, Italy 
This country match >>> Newfoundland and Labrador, Canada >>> Off Newfoundland, Canada 
This country match >>> Lai Châu Province, Vietnam >>> Lai Chau, Vietnam 
This country match >>> Metropolitan France, France >>> Bordeaux, France 
This country match >>> Alabama, United States >>> Off Mobile, Alabama 
This country match >>> Sindh, Pakistan >>> Karachi, Pakistan 
This country match >>> Connecticut, United States >>> Windsor Locks, Connecticut 
This country match >>> India >>> Kalasahar, India 
This country match >>> Vietnam >>> Off Da Nang, Vietnam 
This country match >>> Newfoundland and Labrador, Canada >>> Random Island, Newfoundla

In [10]:
df = data_df
df["Country_Geopy"] = df.index
df["index"] = range(len(df))
df = df.set_index("index")

In [11]:
print(data_df[data_df.Code.apply(lambda x: isinstance(x,float))].Country_Geopy.to_string())

Country_Geopy
Pacific Ocean                  Pacific Ocean
North Pacific Ocean      North Pacific Ocean
Persian Gulf                    Persian Gulf
Indian Ocean                    Indian Ocean
North Pacific Ocean      North Pacific Ocean
Atlantic Ocean                Atlantic Ocean
Baltic Sea                        Baltic Sea
North Sea                          North Sea
North Atlantic Ocean    North Atlantic Ocean
Pacific Ocean                  Pacific Ocean
Atlantic Ocean                Atlantic Ocean
English Channel              English Channel
North Sea                          North Sea
Indian Ocean                    Indian Ocean
Pacific Ocean                  Pacific Ocean
North Atlantic Ocean    North Atlantic Ocean
Atlantic Ocean                Atlantic Ocean
Mediterranean Sea          Mediterranean Sea
Atlantic Ocean                Atlantic Ocean
Sea of Japan                    Sea of Japan
Atlantic Ocean                Atlantic Ocean
Gulf of Tonkin                Gulf of Ton

In [12]:
df.to_csv("data/ready_for_plotting/data_plot_map.csv",index=False)